In [6]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.min_rows', 999)

%config IPCompleter.use_jedi = False

In [7]:
df = pd.read_csv('../data/df_clean_unreg.csv')
df['gr_droppedout_pct'] = df.gr_graduated / df.total
df['gr_graduated_pct'] = df.gr_graduated / df.total
df['gr_iep_pct'] = df.gr_iep / df.total
df['gr_ged_pct'] = df.gr_ged / df.total
df['gr_stillenrolled_pct'] = df.gr_stillenrolled / df.total

dropcols = ["Unnamed: 0", "Unnamed: 0.1", "district_cd", "cohortye", "total", "gr_droppedout", "gr_graduated", "gr_iep", "gr_ged", 
            "gr_stillenrolled", "outcome_yrs", "srcyear", "year", "district_from_df1", "subgroup", "district_from_faru", "districtid", "name", "district_name"]

df_thin = df.drop(columns=dropcols)
df_thin.head()

,num_asian,num_black,num_hisp,num_am_ind,num_white,num_lep,num_free_lunch,num_reduced_lunch,num_multi,ufb,staid,star,totstat,lrev,fedrev,totrev,state_aid_total_revenues,star_total_revenues,state_revenues_total_revenues,bded,cadm,tsal,pps,cds,boc,tui1,tui2,ois,oiexp,cser,opmt,tchr,heal,oeb,total_fringe_benefits,ound,oth,subtot,trans,dsp,dsi,texp,total_unexpended_surplus_funds_t,revenue_expenditures,var,dcaadm,rev_pupil,exp_pupil,av_twpu,inc_twpu,pwr,apwr,cwr,instructional_salaries,instructional_fringe_benefits,ins_fb,adjusted_expenditures__subtotal,ie1,ie1_total,ie2_instructional_expenditures_i,ie2,ie3_instructional_expenditures_i,ie3_adj_exp,loc_eff_rate,county_name,gr_droppedout_pct,gr_graduated_pct,gr_iep_pct,gr_ged_pct,gr_stillenrolled_pct
0,269.0,5952.0,918.0,21.0,1884.0,300.0,5505.0,970.0,0.0,5680290,54570126,10225351,64795477,98285968,16659701,179741146,0.303604,0.056889,0.360493,490999,2118256,54023846,5251097,4501195,3728010,188593,6610228,16305802,19303934,1199428,11203660,5165294,14257866,14441655,33864815,4053261,564400,163407524,5972182,7669294,5365766,182414766,0.031139,-2673620,-0.014657,10559,17022.553840,17275.761530,244659,93587,0.706290,0.789764,0.748027,83809950,3.117730e+07,0.170914,162986931,109912705.0,0.602543,1.410900e+08,0.773457,1.409014e+08,0.864495,26.51,Albany,0.658576,0.658576,0.033981,0.092233,0.080906
1,360.0,5281.0,940.0,24.0,1654.0,382.0,3992.0,789.0,77.0,13358464,74675977,10131780,84807757,105784891,13677063,204269711,0.365575,0.049600,0.415175,385910,2055778,55114952,5988074,4266372,4793593,541340,7253219,17557992,26611527,602434,11451223,5439024,16966397,11086334,33491755,3264726,989999,174368894,6153173,7356028,6733231,194611326,0.068642,9658385,0.049629,10430,19584.823682,18658.804027,268181,99117,0.628353,0.725600,0.676977,87720983,3.087779e+07,0.158664,173595554,122127069.0,0.627543,1.530049e+08,0.786207,1.524635e+08,0.878269,21.95,Albany,0.663212,0.663212,0.069085,0.046632,0.048359
2,360.0,5281.0,940.0,24.0,1654.0,382.0,3992.0,789.0,77.0,13358464,74675977,10131780,84807757,105784891,13677063,204269711,0.365575,0.049600,0.415175,385910,2055778,55114952,5988074,4266372,4793593,541340,7253219,17557992,26611527,602434,11451223,5439024,16966397,11086334,33491755,3264726,989999,174368894,6153173,7356028,6733231,194611326,0.068642,9658385,0.049629,10430,19584.823682,18658.804027,268181,99117,0.628353,0.725600,0.676977,87720983,3.087779e+07,0.158664,173595554,122127069.0,0.627543,1.530049e+08,0.786207,1.524635e+08,0.878269,21.95,Albany,0.564767,0.564767,0.051813,0.044905,0.193437
3,360.0,5281.0,940.0,24.0,1654.0,382.0,3992.0,789.0,77.0,13358464,74675977,10131780,84807757,105784891,13677063,204269711,0.365575,0.049600,0.415175,385910,2055778,55114952,5988074,4266372,4793593,541340,7253219,17557992,26611527,602434,11451223,5439024,16966397,11086334,33491755,3264726,989999,174368894,6153173,7356028,6733231,194611326,0.068642,9658385,0.049629,10430,19584.823682,18658.804027,268181,99117,0.628353,0.725600,0.676977,87720983,3.087779e+07,0.158664,173595554,122127069.0,0.627543,1.530049e+08,0.786207,1.524635e+08,0.878269,21.95,Albany,0.685665,0.685665,0.070812,0.050086,0.015544
4,382.0,4894.0,908.0,27.0,1592.0,405.0,4660.0,757.0,96.0,11443927,88986207,9683819,98670026,108582740,12186499,219439265,0.405516,0.044130,0.449646,325611,2333872,57538745,6323772,4632120,5340457,734445,7801336,18044340,30170560,428795,11628871,5143395,17442327,13876660,36462382,3527941,760170,186053417,6320002,8410609,6874804,207658832,0.055109,11780433,0.056730,10702,20504.509905,19403.740609,310466,103451,0.650327,0.694768,0.672548,91879434,3.307660e+07,0.159283,185202972,130585775.0,0.628800,1.636624e+08,0.788131,1.629279e+08,0.879726,21.25,Albany,0.646179,0.646179,0.036545,0.024917,0.054817


In [8]:
df.tail(20).to_csv('/Users/dave/src/CSE6242/dave/small_headers.csv')

# PCA, Modeling and Predictions by County

In [9]:
%%time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score


results = {}
county_agg_results = pd.DataFrame()

def train_model(X, y, test_size=0.30, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    m1 = regr.coef_[0]
    m2 = regr.coef_[1]
    c = regr.intercept_
    return(y_pred, mse, r2, m1, m2, c)

i = 0
imp = {}

for county in df_thin.county_name.unique():
    county_df = df_thin[df_thin.county_name == county].drop(columns = ['county_name', 'num_am_ind'])
    X = county_df.drop(["gr_droppedout_pct", "gr_graduated_pct", "gr_iep_pct", "gr_ged_pct", "gr_stillenrolled_pct"], axis=1, inplace=False)
    y = pd.DataFrame({'y':county_df['gr_graduated_pct']}).reset_index()
    x = StandardScaler().fit_transform(X)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDF = pd.DataFrame(data = principalComponents, 
                               columns = ['X1', 'X2'])
    
    
    n_pcs= pca.components_.shape[0]
    
    # get most important features:
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = list(X.columns)
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

    # build the dataframe
    imp_df = pd.DataFrame(dic.items())
    imp[county] = imp_df
    
    
    raw_df = pd.concat([principalDF, y.y], axis=1)
    y_pred, mse, r2, m1, m2, c = train_model(X = principalDF, y = y.y)
    
    raw_df_pred = pd.concat([raw_df, pd.DataFrame(y_pred)], axis=1)
    
    raw_df_pred.rename(columns={0:'y_pred'}, inplace=True)
    results[county] = raw_df_pred    
    
    agg_results = pd.DataFrame({'EV': pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1], 
                                'MSE': mse,
                                'R2': r2, 
                                'M1': m1,
                                'M2': m2,
                                'C': c}, 
                                index=[county])
    county_agg_results = county_agg_results.append(agg_results)
    i = i+1

# X1 - PCA1
# X2 - PCA2
# Y - gr_graduated_pct
# Pred Y () # this can be any model (linear regression + line of best fit formula (so C & M Parameters))
# County

CPU times: user 5.38 s, sys: 3.89 s, total: 9.27 s
Wall time: 1.75 s


# PCA Most Important Feature by County
It should be noted that these are mostly revenue and expenditure based.

In [21]:
imp

{'Albany':      0                                1
 0  PC0  adjusted_expenditures__subtotal
 1  PC1                              cwr,
 'Schoharie':      0       1
 0  PC0  totrev
 1  PC1     cwr,
 'Greene':      0          1
 0  PC0        ie1
 1  PC1  exp_pupil,
 'Schenectady':      0                                 1
 0  PC0  ie2_instructional_expenditures_i
 1  PC1     state_revenues_total_revenues,
 'Allegany':      0                                1
 0  PC0  adjusted_expenditures__subtotal
 1  PC1                              cwr,
 'Steuben':      0                                1
 0  PC0  adjusted_expenditures__subtotal
 1  PC1                              cwr,
 'Livingston':      0                                1
 0  PC0  adjusted_expenditures__subtotal
 1  PC1                              cwr,
 'Wyoming':      0                                1
 0  PC0  adjusted_expenditures__subtotal
 1  PC1                         inc_twpu,
 'Cattaraugus':      0                            

In [10]:
# flatten dict to df

final_results = pd.DataFrame()

for k, v in results.items():
    df = v
    df.loc[:, 'county'] = k
    final_results = pd.concat([df, final_results], 0)

# final_results.columns = ['Unnamed: 0', 'X1', 'X2', 'y', 'y_pred', 'county']

In [11]:
final_results.head()

,X1,X2,y,y_pred,county
0,-0.427394,-0.048193,0.883901,0.907400,Suffolk
1,0.316367,0.643116,0.978836,0.899511,Suffolk
2,0.316367,0.643116,0.955908,0.848542,Suffolk
3,0.316367,0.643116,0.978836,0.901062,Suffolk
4,0.521213,1.308565,0.923875,0.873801,Suffolk


In [12]:
county_agg_results.head()

,EV,MSE,R2,M1,M2,C
Albany,0.717430,0.002786,0.702530,-0.005648,0.025772,0.845356
Schoharie,0.635063,0.004321,0.070491,-0.002086,0.006593,0.830155
Greene,0.713559,0.003910,0.282484,-0.003642,0.010040,0.825641
Schenectady,0.715885,0.003802,0.744846,-0.012521,0.022741,0.832585
Allegany,0.632762,0.005167,0.106989,-0.001569,0.007430,0.851881


In [13]:
import pickle
# pickle.dump(final_results, open("../data/PCA_plus_preds.pkl", 'wb'))
# pickle.dump(county_agg_results, open("../data/county_agg_results.pkl", 'wb'))
final_results.to_csv('../data/PCA_plus_preds.csv')
county_agg_results.to_csv("../data/county_agg_results.csv")

In [14]:
a = pd.read_csv("../data/PCA_plus_preds.csv")
b = pd.read_csv("../data/county_agg_results.csv")

In [15]:
display(a.head())
display(b.head())

,Unnamed: 0,X1,X2,y,y_pred,county
0,0,-0.427394,-0.048193,0.883901,0.907400,Suffolk
1,1,0.316367,0.643116,0.978836,0.899511,Suffolk
2,2,0.316367,0.643116,0.955908,0.848542,Suffolk
3,3,0.316367,0.643116,0.978836,0.901062,Suffolk
4,4,0.521213,1.308565,0.923875,0.873801,Suffolk


,Unnamed: 0,EV,MSE,R2,M1,M2,C
0,Albany,0.717430,0.002786,0.702530,-0.005648,0.025772,0.845356
1,Schoharie,0.635063,0.004321,0.070491,-0.002086,0.006593,0.830155
2,Greene,0.713559,0.003910,0.282484,-0.003642,0.010040,0.825641
3,Schenectady,0.715885,0.003802,0.744846,-0.012521,0.022741,0.832585
4,Allegany,0.632762,0.005167,0.106989,-0.001569,0.007430,0.851881


In [16]:
# df_thin.groupby('county_name').count()
b.R2.mean()

0.27747936785574223

In [17]:
# df.tail(20).to_csv('/Users/dave/src/CSE6242/dave/small_headers.csv')

In [18]:
df.tail(20)

,X1,X2,y,y_pred,county
554,-4.587500,-2.784567,0.963077,NaN,Suffolk
555,-4.587500,-2.784567,0.954407,NaN,Suffolk
556,-4.363919,-1.804936,0.975155,NaN,Suffolk
557,-4.363919,-1.804936,0.960961,NaN,Suffolk
558,-4.363919,-1.804936,0.910299,NaN,Suffolk
559,-3.949911,-2.032501,0.943709,NaN,Suffolk
560,-3.949911,-2.032501,0.961078,NaN,Suffolk
561,-3.949911,-2.032501,0.941176,NaN,Suffolk
562,-3.804432,-1.700880,0.960784,NaN,Suffolk
563,-3.804432,-1.700880,0.950495,NaN,Suffolk


In [19]:
df.head()

,X1,X2,y,y_pred,county
0,-0.427394,-0.048193,0.883901,0.907400,Suffolk
1,0.316367,0.643116,0.978836,0.899511,Suffolk
2,0.316367,0.643116,0.955908,0.848542,Suffolk
3,0.316367,0.643116,0.978836,0.901062,Suffolk
4,0.521213,1.308565,0.923875,0.873801,Suffolk


# Feature Importance

In [128]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import Lasso
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression





def lasso_me(X, y, alphas, names, test_size=0.30, random_state=42):
    results_df = pd.DataFrame()
    results_df['Feature Name'] = names
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    for alpha in alphas:
#         temp_df = pd.DataFrame()
        L2 = Lasso(alpha=alpha, 
                   max_iter=10000,
                   fit_intercept=False)
#         cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#         # evaluate model
#         scores = cross_val_score(L2, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#         # force scores to be positive
#         scores = absolute(scores)
        L2.fit(X_train, y_train)
        column_name = 'Alpha = %f' % alpha
        results_df[column_name] = L2.coef_
    return(results_df)

county_lasso = {}

for county in df_thin.county_name.unique():
    county_df = df_thin[df_thin.county_name == county].drop(columns = ['county_name', 'num_am_ind'])
    X = county_df.drop(["gr_droppedout_pct", "gr_graduated_pct", "gr_iep_pct", "gr_ged_pct", "gr_stillenrolled_pct"], axis=1, inplace=False)
    y = pd.DataFrame({'y':county_df['gr_graduated_pct']}).reset_index()['y']

    x = StandardScaler().fit_transform(X)
    alphas = [0.0001, 0.001, 0.01]
    county_lasso[county] = lasso_me(x, y, 
                                    alphas = alphas, 
                                    names=X.columns, 
                                    test_size=0.30, 
                                    random_state=42)
import warnings
warnings.filterwarnings('ignore')

In [131]:
def plot_assumptions(X):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 16))
#     # generate OLS model
#     # fit multiple linear regression model
#     model = LinearRegression()
#     model_fit = model.fit(X, y)
#     r_sq = model_fit.score(X, y)
#     sns.regplot(X, y)
    return(None)

# plot_assumptions(X)

# Feature Importance through Lasso Selection
Note: I don't love these results because we didn't validate typical assumptions of homoskedasticity, linearity, independence, etc.  They are, however, foundationally interesting for other types of studies, so I'm leaving them in here:

-dave

In [112]:
for county in df_thin.county_name.unique():
    print("\n==================== " + county + "========================")
    display(county_lasso[county].sort_values(by=['Alpha = %f' % a for a in alphas], ascending=False).head(3))


==================== Albany========================


,Feature Name,Alpha = 0.000000,Alpha = 0.000010,Alpha = 0.000100,Alpha = 0.001000
4,num_lep,0.082683,0.078059,0.050122,0.0
45,rev_pupil,0.080694,0.065945,0.000000,-0.0
26,ois,0.079953,0.077499,0.046569,0.0



==================== Schoharie========================


,Feature Name,Alpha = 0.000000,Alpha = 0.000010,Alpha = 0.000100,Alpha = 0.001000
26,ois,0.061520,0.061444,0.046159,0.005396
44,dcaadm,0.060755,0.056701,0.044845,0.000000
48,inc_twpu,0.055771,0.055715,0.053095,0.022573



==================== Greene========================


...

,Feature Name,Alpha = 0.000000,Alpha = 0.000010,Alpha = 0.000100,Alpha = 0.001000
47,av_twpu,0.074981,0.067215,0.011296,-0.0
10,star,0.061426,0.058903,0.024400,0.0
9,staid,0.056286,0.043948,-0.000000,-0.0



==================== Rockland========================


,Feature Name,Alpha = 0.000000,Alpha = 0.000010,Alpha = 0.000100,Alpha = 0.001000
47,av_twpu,0.019828,0.019827,0.015712,0.006016
15,state_aid_total_revenues,0.019651,0.011812,0.008833,0.000000
44,dcaadm,0.015992,0.022420,0.000000,0.000000



==================== Suffolk========================


,Feature Name,Alpha = 0.000000,Alpha = 0.000010,Alpha = 0.000100,Alpha = 0.001000
3,num_white,0.054654,0.054105,0.028918,0.004712
0,num_asian,0.049517,0.048573,0.033027,0.008684
37,trans,0.043363,0.042762,0.018195,0.000000
